In [ ]:
import sys
sys.path.insert(0, "..")
from dalgebra import *
from dalgebra.commutators.almost_commuting import almost_commuting_wilson, generic_normal
from functools import lru_cache
import logging

logger = logging.getLogger("dalgebra")

In [6]:
@lru_cache
def recursive_computation(n: int, m:int):
    if m <= 0:
        raise ValueError("Value for {m} must be positive")
    elif m%n == 0:
        L = generic_normal(n)
        return (L.sym_power(m//n), tuple(L.parent().zero() for _ in range(n-1)))
    elif m < n:
        return almost_commuting_wilson(n,m)
    else: # recursive case
        logger.debug(f"Computing recursion case ({n=}, m={m-n})...")
        P, T = recursive_computation(n, m-n)
        R = P.parent()
        L = generic_normal(n, output_ring=R)
        z = R.gen("z")
        logger.debug(f"Computing [L, P_{m-n}]...")
        iT = T; T = sum(T[i]*z[i] for i in range(len(T)))
        
        ## P is the ´m-n´ element of the basis
        ## [L, P] = T
        logger.debug(f"Extending variables...")
        ER = R.append_variables(*[f"y_{i}" for i in range(n)])
        ## Fast conversion because ´ER´ was built using "append_variables" 
        oL, L = L, ER(L); oP, P = P, ER(P); oT, T = T, ER(T)
        oz, z = z, ER.gen("z")
        logger.debug(f"Creating new objects...")
        y = [ER.gen(f"y_{i}") for i in range(n)]
        Q = sum(y[i]*z[i] for i in range(n))
        
        ## We want that L*P + Q is the element in the almost commuting basis.
        ## [L, L*P + Q] = L*[L, P] + [L, Q] = L*T + [L,Q]
        logger.debug(f"Computing [L, L*P_{m} + Q]...")
        com_LQ = L.lie_bracket(Q, z)
        com = L(z=T) + com_LQ
        logger.debug(f"Getting the differential equations...")
        equs = [com.coefficient_full(z[i]) for i in range(n-1, com.order(z)+1)]
        system = DSystem(equs, variables=y)
        logger.debug(f"Computing the solution to the system:\n{system}")
        solution = system.solve_linear()
        
        ## Plugging solution
        logger.debug(f"Plugging computed solution:\n{solution}")
        nP = oL(z=oP) + sum(R(solution[y[i]])*oz[i] for i in range(n))
        nT = tuple(R(com.coefficient_full(z[i])(dic=solution)) for i in range(n-1))
                   
        logger.debug(f"Finished execution for {n=} and {m=}")
        return nP, nT      

In [7]:
%time P1, T1 = recursive_computation(3, 20)

KeyboardInterrupt: 

In [5]:
%time P2, T2 = almost_commuting_wilson(3,20)

CPU times: user 5min 5s, sys: 110 ms, total: 5min 6s
Wall time: 5min 6s


In [7]:
T1 == T2

True

In [2]:
from dalgebra.commutators.almost_commuting import recursion